In [1]:
import xlrd
import os
import pandas as pd
import numpy as np
import math

In [2]:
cwd = os.getcwd()
cwd

'/Volumes/GoogleDrive/My Drive/STRIDE/pml - Biweekly change report'

In [3]:
os.chdir("/Volumes/GoogleDrive/My Drive/STRIDE/pml - Biweekly change report")

In [4]:
os.listdir('.')

['0520 Property Master List.xlsx',
 '0603 Property Master List.xlsx',
 'output',
 '.ipynb_checkpoints',
 '0422 Property Master List.xlsx',
 '0506 Property Master List.xlsx',
 'archived',
 'track change 1.R',
 '.Rhistory',
 '.RData',
 '0701 Property Master List.xlsx',
 'old_new.xlsx',
 'email version',
 '0715 Property Master List.xlsx',
 'MoM change analysis',
 '0729 Property Master List.xlsx',
 'Biweekly Change Reason.gdoc',
 '0812 Property Master List.xlsx',
 'Biweekly change check.ipynb',
 '0826 Property Master List.xlsx']

# Read files

In [5]:
old1=pd.read_excel('0812 Property Master List.xlsx')

In [6]:
new1=pd.read_excel('0826 Property Master List.xlsx')

# Compare difference

## 1. substract the variables need to be compared:
Capacity
Status
Change reason: New site, Decom, Expansion, Downsizing, 

In [7]:
old1.shape

(1386, 18)

In [8]:
new1.shape

(1394, 18)

In [9]:
list(new1.columns)

['Property Code',
 'Status',
 'Property Type (Building)',
 'Property Use Sub Type',
 'Address',
 'Address 2',
 'City',
 'Zip/Postal Code',
 'State/Province',
 'ISO-3 Country Code',
 'Country',
 'Capacity - Bldg',
 'RSF',
 'Property Description',
 'Region - Original',
 'Region',
 'Date Opened',
 'Date Closed']

In [10]:
new=new1.drop(columns=['Property Type (Building)',
 'Property Use Sub Type',
 'Address',
 'Address 2',
 'City',
 'Zip/Postal Code',
 'State/Province',
 'ISO-3 Country Code',
 'Country','Property Description',
 'Region - Original',
 'Region','Date Opened',
 'Date Closed' ])

In [11]:
old=old1.drop(columns = ['Property Type (Building)',
 'Property Use Sub Type',
 'Address',
 'Address 2',
 'City',
 'Zip/Postal Code',
 'State/Province',
 'ISO-3 Country Code',
 'Country','Property Description',
 'Region - Original',
 'Region','Date Opened',
 'Date Closed'])

In [12]:
list(new.columns)

['Property Code', 'Status', 'Capacity - Bldg', 'RSF']

In [ ]:
df_merged = pd.concat([old.set_index('Property Code'), new.set_index('Property Code')], 
                   axis='columns', keys=['Old', 'New'], sort = True)
df_all = df_merged.swaplevel(axis='columns')[old.columns[1:]]

df_all.head()

In [ ]:
def highlight_diff(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Old', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

#df_all.style.apply(highlight_diff, axis=None)


In [ ]:
df_all.to_excel("old_new.xlsx")

## method 1

In [13]:
#1.check the difference with NaN values
merged = pd.merge(old, new, on = 'Property Code', indicator= True, how = 'outer')
merged.columns = ['Property Code', 'Former Status','Former Capacity','Former RSF','Status','Capacity','RSF','merge']


In [14]:
old_only = merged[merged['merge'] == 'right_only']
old_only

,Property Code,Former Status,Former Capacity,Former RSF,Status,Capacity,RSF,merge
1386,YYZ_3660,NaN,NaN,NaN,Closed,4,600.0,right_only
1387,YGB_542C,NaN,NaN,NaN,Future,0,0.0,right_only
1388,SDQ_MM,NaN,NaN,NaN,Future,0,0.0,right_only
1389,VLC_CGC9,NaN,NaN,NaN,Open,5,500.0,right_only
1390,GTM_278,NaN,NaN,NaN,Future,0,0.0,right_only
1391,FCO_VO92,NaN,NaN,NaN,Future,0,0.0,right_only
1392,BKK_111S,NaN,NaN,NaN,Open,0,0.0,right_only
1393,RDV_300M,NaN,NaN,NaN,Open,2,400.0,right_only


In [15]:
new_only = merged[merged['merge'] == 'left_only']
new_only

,Property Code,Former Status,Former Capacity,Former RSF,Status,Capacity,RSF,merge


In [ ]:
#no records only in new file

In [ ]:
#2.for the same records, check the difference

In [16]:
list(merged.columns)

['Property Code',
 'Former Status',
 'Former Capacity',
 'Former RSF',
 'Status',
 'Capacity',
 'RSF',
 'merge']

In [17]:
merged.dtypes

Property Code        object
Former Status        object
Former Capacity     float64
Former RSF          float64
Status               object
Capacity              int64
RSF                 float64
merge              category
dtype: object

In [18]:
merged['Capacity']=merged['Capacity'].astype('float64')


In [19]:
merged.head()

,Property Code,Former Status,Former Capacity,Former RSF,Status,Capacity,RSF,merge
0,AUH_SKY,Closed,0.0,0.0,Closed,0.0,0.0,both
1,DXB_A007,Open,96.0,10736.0,Open,96.0,10736.0,both
2,DXB_DIC,Open,6.0,2233.0,Open,6.0,2233.0,both
3,AEP_160C,Closed,0.0,0.0,Closed,0.0,0.0,both
4,AEP_1000,Open,62.0,4000.0,Open,62.0,4000.0,both


In [20]:
conditions = [
    
    (pd.isnull(merged['Former Status'])|merged['Former Status'].str.contains('Future')) & merged['Status'].str.contains('Open'),
    (merged['Status'].str.contains('Closed') & merged['Former Status'].str.contains('Open')),   
    (merged['Former Capacity'] > merged['Capacity']) | (merged['Former RSF'] > merged['RSF']) , 
    (merged['Former Capacity'] < merged['Capacity']) | (merged['Former RSF'] < merged['RSF'])
]

choices = ['New Site','Decom','Downsizing','Expansion']

merged['Change reason'] = np.select(conditions, choices, default="")

In [21]:
change1=merged[merged['Change reason'].str.contains('Downsizing|Expansion|New site|Decom')]
change1

,Property Code,Former Status,Former Capacity,Former RSF,Status,Capacity,RSF,merge,Change reason
186,SOD_S287,Open,3.0,517.0,Closed,3.0,517.0,both,Decom
691,NRT_1251,Open,45.0,1494.0,Closed,45.0,1494.0,both,Decom
1272,SFO_130P,Open,0.0,0.0,Closed,0.0,0.0,both,Decom


In [22]:
total=pd.concat([change1,old_only], axis=0, ignore_index=True,sort=False)

In [23]:
total

,Property Code,Former Status,Former Capacity,Former RSF,Status,Capacity,RSF,merge,Change reason
0,SOD_S287,Open,3.0,517.0,Closed,3.0,517.0,both,Decom
1,NRT_1251,Open,45.0,1494.0,Closed,45.0,1494.0,both,Decom
2,SFO_130P,Open,0.0,0.0,Closed,0.0,0.0,both,Decom
3,YYZ_3660,NaN,NaN,NaN,Closed,4.0,600.0,right_only,NaN
4,YGB_542C,NaN,NaN,NaN,Future,0.0,0.0,right_only,NaN
5,SDQ_MM,NaN,NaN,NaN,Future,0.0,0.0,right_only,NaN
6,VLC_CGC9,NaN,NaN,NaN,Open,5.0,500.0,right_only,NaN
7,GTM_278,NaN,NaN,NaN,Future,0.0,0.0,right_only,NaN
8,FCO_VO92,NaN,NaN,NaN,Future,0.0,0.0,right_only,NaN
9,BKK_111S,NaN,NaN,NaN,Open,0.0,0.0,right_only,NaN


In [24]:
result = total.drop_duplicates(subset='Property Code', keep='first')
result

,Property Code,Former Status,Former Capacity,Former RSF,Status,Capacity,RSF,merge,Change reason
0,SOD_S287,Open,3.0,517.0,Closed,3.0,517.0,both,Decom
1,NRT_1251,Open,45.0,1494.0,Closed,45.0,1494.0,both,Decom
2,SFO_130P,Open,0.0,0.0,Closed,0.0,0.0,both,Decom
3,YYZ_3660,NaN,NaN,NaN,Closed,4.0,600.0,right_only,NaN
4,YGB_542C,NaN,NaN,NaN,Future,0.0,0.0,right_only,NaN
5,SDQ_MM,NaN,NaN,NaN,Future,0.0,0.0,right_only,NaN
6,VLC_CGC9,NaN,NaN,NaN,Open,5.0,500.0,right_only,NaN
7,GTM_278,NaN,NaN,NaN,Future,0.0,0.0,right_only,NaN
8,FCO_VO92,NaN,NaN,NaN,Future,0.0,0.0,right_only,NaN
9,BKK_111S,NaN,NaN,NaN,Open,0.0,0.0,right_only,NaN


In [25]:
result['Change reason'] = result['Change reason'].fillna('New Site')
result['Former Capacity']=result['Former Capacity'].fillna(0)
result['Former RSF']=result['Former RSF'].fillna(0)


In [26]:
result

,Property Code,Former Status,Former Capacity,Former RSF,Status,Capacity,RSF,merge,Change reason
0,SOD_S287,Open,3.0,517.0,Closed,3.0,517.0,both,Decom
1,NRT_1251,Open,45.0,1494.0,Closed,45.0,1494.0,both,Decom
2,SFO_130P,Open,0.0,0.0,Closed,0.0,0.0,both,Decom
3,YYZ_3660,NaN,0.0,0.0,Closed,4.0,600.0,right_only,New site
4,YGB_542C,NaN,0.0,0.0,Future,0.0,0.0,right_only,New site
5,SDQ_MM,NaN,0.0,0.0,Future,0.0,0.0,right_only,New site
6,VLC_CGC9,NaN,0.0,0.0,Open,5.0,500.0,right_only,New site
7,GTM_278,NaN,0.0,0.0,Future,0.0,0.0,right_only,New site
8,FCO_VO92,NaN,0.0,0.0,Future,0.0,0.0,right_only,New site
9,BKK_111S,NaN,0.0,0.0,Open,0.0,0.0,right_only,New site


In [27]:
result=result.drop(columns=['merge'])

In [ ]:
## merge with other info, 

In [28]:
result['RSF Change']=result['RSF']- result['Former RSF']
result['Capacity Change']=result['Capacity']- result['Former Capacity']

In [29]:
result

,Property Code,Former Status,Former Capacity,Former RSF,Status,Capacity,RSF,Change reason,RSF Change,Capacity Change
0,SOD_S287,Open,3.0,517.0,Closed,3.0,517.0,Decom,0.0,0.0
1,NRT_1251,Open,45.0,1494.0,Closed,45.0,1494.0,Decom,0.0,0.0
2,SFO_130P,Open,0.0,0.0,Closed,0.0,0.0,Decom,0.0,0.0
3,YYZ_3660,NaN,0.0,0.0,Closed,4.0,600.0,New site,600.0,4.0
4,YGB_542C,NaN,0.0,0.0,Future,0.0,0.0,New site,0.0,0.0
5,SDQ_MM,NaN,0.0,0.0,Future,0.0,0.0,New site,0.0,0.0
6,VLC_CGC9,NaN,0.0,0.0,Open,5.0,500.0,New site,500.0,5.0
7,GTM_278,NaN,0.0,0.0,Future,0.0,0.0,New site,0.0,0.0
8,FCO_VO92,NaN,0.0,0.0,Future,0.0,0.0,New site,0.0,0.0
9,BKK_111S,NaN,0.0,0.0,Open,0.0,0.0,New site,0.0,0.0


In [30]:
list(result.columns)

['Property Code',
 'Former Status',
 'Former Capacity',
 'Former RSF',
 'Status',
 'Capacity',
 'RSF',
 'Change reason',
 'RSF Change',
 'Capacity Change']

In [31]:
cols=['Property Code','Change reason','Former RSF','RSF Change',
      'Former Capacity','Capacity Change','Status','Former Status', 'Capacity','RSF']

In [32]:
result1=result[cols]
result1=result1.drop(columns=['Status','Capacity','RSF'])

In [33]:
result1


,Property Code,Change reason,Former RSF,RSF Change,Former Capacity,Capacity Change,Former Status
0,SOD_S287,Decom,517.0,0.0,3.0,0.0,Open
1,NRT_1251,Decom,1494.0,0.0,45.0,0.0,Open
2,SFO_130P,Decom,0.0,0.0,0.0,0.0,Open
3,YYZ_3660,New site,0.0,600.0,0.0,4.0,NaN
4,YGB_542C,New site,0.0,0.0,0.0,0.0,NaN
5,SDQ_MM,New site,0.0,0.0,0.0,0.0,NaN
6,VLC_CGC9,New site,0.0,500.0,0.0,5.0,NaN
7,GTM_278,New site,0.0,0.0,0.0,0.0,NaN
8,FCO_VO92,New site,0.0,0.0,0.0,0.0,NaN
9,BKK_111S,New site,0.0,0.0,0.0,0.0,NaN


In [34]:
output=pd.merge(new1,result1, on ='Property Code', how = 'inner')

In [35]:
output

,Property Code,Status,Property Type (Building),Property Use Sub Type,Address,Address 2,City,Zip/Postal Code,State/Province,ISO-3 Country Code,...,Region - Original,Region,Date Opened,Date Closed,Change reason,Former RSF,RSF Change,Former Capacity,Capacity Change,Former Status
0,SOD_S287,Closed,Retail,GL Hub,"Rua Sete de Setembro, 287","5 andar, Sala 52",Sorocaba,18035-001,Sao Paulo,BRA,...,LATAM - Brazil,LATAM,2016-09-12,2019-05-10,Decom,517.0,0.0,3.0,0.0,Open
1,YYZ_3660,Closed,Coworking,Office - Coworking,3660 Midland Avenue,Suite 314,Ontario,M1V 0B8,ON,CAN,...,US&C - US Central & Canada,US&C,2016-02-01,2016-07-31,New site,0.0,600.0,0.0,4.0,NaN
2,YGB_542C,Future,Retail,GL Spot,542 Charest East Boulevard,NaN,Quebec City,G1K 9K6,QC,CAN,...,US&C - US Central & Canada,US&C,NaT,NaT,New site,0.0,0.0,0.0,0.0,NaN
3,SDQ_MM,Future,Retail,GL Spot,"Avenida San Vicente de Paúl, Megacentro Mall C...",NaN,Santo Domingo Este,NaN,NaN,DOM,...,LATAM - ACAC,LATAM,NaT,NaT,New site,0.0,0.0,0.0,0.0,NaN
4,VLC_CGC9,Open,Coworking,Eats,"Carrer de Guillem de Castro, 9",Office 14 & 15,Valencia,46007,NaN,ESP,...,EMEA - WSE,EMEA,2019-08-01,NaT,New site,0.0,500.0,0.0,5.0,NaN
5,GTM_278,Future,Retail,GL Spot,19 Avenida 278,NaN,Guatemala,NaN,NaN,GTM,...,LATAM - ACAC,LATAM,NaT,NaT,New site,0.0,0.0,0.0,0.0,NaN
6,FCO_VO92,Future,Coworking,Office & GL,"Via Ostiense, 92","Offices 003, 101-104",Rome,154,NaN,ITA,...,EMEA - WSE,EMEA,NaT,NaT,New site,0.0,0.0,0.0,0.0,NaN
7,NRT_1251,Closed,Retail,GL Hub,"1-25-1, Ebisu, Shibuya - ku, Nagahama Building...",NaN,Tokyo,150-0013,NaN,JPN,...,APAC - NA,APAC,2017-05-01,2019-08-08,Decom,1494.0,0.0,45.0,0.0,Open
8,BKK_111S,Open,Virtual,Office - Virtual,11/1 South Sathorn Road,"9th & 10th Floor, AIA Sathorn Towe",Yannawa,10120,NaN,THA,...,APAC - Grab,APAC,2019-08-26,NaT,New site,0.0,0.0,0.0,0.0,NaN
9,RDV_300M,Open,Coworking,Office - Coworking,300 Morris St,7th Floor,Durham,27701,NC,USA,...,US&C - US East,US&C,2019-08-19,NaT,New site,0.0,400.0,0.0,2.0,NaN


In [36]:
output = output.sort_values(by=['Change reason', 'Region','Property Code'])
output1 =output.drop(columns=['Date Opened','Former Status'])

In [37]:
output1.to_excel("/Volumes/GoogleDrive/My Drive/STRIDE/pml - Biweekly change report/output/biweekly change 0826.xlsx")